# ETL Pipeline with Python and Google BigQuery

This notebook demonstrates a complete ETL process (Extract–Transform–Load) using Python, pandas, and Google BigQuery.  
It shows how to authenticate with Google Cloud, process a dataset, load it into a BigQuery table, and validate the result using SQL.

---


In [7]:
# ============================================================
# 1. INITIAL SETUP
# ============================================================

import os
import pandas as pd
from google.cloud import bigquery
from google.auth import load_credentials_from_file

# Path to credentials
creds_path = os.path.join("credentials", "service_account.json")

if not os.path.isfile(creds_path):
    raise FileNotFoundError(
        f"Credentials file not found at:\n{creds_path}"
    )

# Google Cloud project
project_id = "eloquent-hangar-474417-t1"

# Initialize BigQuery client
creds, _ = load_credentials_from_file(creds_path)
client = bigquery.Client(credentials=creds, project=project_id)

print("BigQuery client successfully initialized.")


BigQuery client successfully initialized.


In [8]:
# ============================================================
# 2. EXTRACT — Load dataset from remote repository
# ============================================================

url = "https://raw.githubusercontent.com/mwaskom/seaborn-data/master/iris.csv"

df_raw = pd.read_csv(url)

print("Rows / Columns:", df_raw.shape)
df_raw.head()


Rows / Columns: (150, 5)


,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa
2,4.7,3.2,1.3,0.2,setosa
3,4.6,3.1,1.5,0.2,setosa
4,5.0,3.6,1.4,0.2,setosa


In [9]:
# ============================================================
# 3. TRANSFORM — Clean and aggregate data
# ============================================================

df_transformed = df_raw.groupby("species", as_index=False).mean()

df_transformed


,species,sepal_length,sepal_width,petal_length,petal_width
0,setosa,5.006,3.428,1.462,0.246
1,versicolor,5.936,2.770,4.260,1.326
2,virginica,6.588,2.974,5.552,2.026


In [10]:
# ============================================================
# 4. LOAD — Upload transformed data to BigQuery
# ============================================================

dataset_id = "demo_dataset"
table_id = "iris_summary"

table_ref = f"{project_id}.{dataset_id}.{table_id}"

# Load to BigQuery
job = client.load_table_from_dataframe(df_transformed, table_ref)
job.result()

print(f"Table loaded successfully: {table_ref}")


c:\Users\casco\AppData\Local\Programs\Python\Python312\Lib\site-packages\google\cloud\bigquery\_pandas_helpers.py:484: FutureWarning: Loading pandas DataFrame into BigQuery will require pandas-gbq package version 0.26.1 or greater in the future. Tried to import pandas-gbq and got: No module named 'pandas_gbq'
  warnings.warn(


Table loaded successfully: eloquent-hangar-474417-t1.demo_dataset.iris_summary


In [11]:
# ============================================================
# 5. VALIDATION — Query table from BigQuery
# ============================================================

query = f"SELECT * FROM `{table_ref}` LIMIT 10"
df_check = client.query(query).to_dataframe()

df_check


c:\Users\casco\AppData\Local\Programs\Python\Python312\Lib\site-packages\google\cloud\bigquery\table.py:1994: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


,species,sepal_length,sepal_width,petal_length,petal_width
0,setosa,5.006,3.428,1.462,0.246
1,versicolor,5.936,2.770,4.260,1.326
2,virginica,6.588,2.974,5.552,2.026
3,setosa,5.006,3.428,1.462,0.246
4,versicolor,5.936,2.770,4.260,1.326
5,virginica,6.588,2.974,5.552,2.026
6,setosa,5.006,3.428,1.462,0.246
7,setosa,5.006,3.428,1.462,0.246
8,setosa,5.006,3.428,1.462,0.246
9,setosa,5.006,3.428,1.462,0.246


# Conclusion

The ETL pipeline ran successfully:

- **Extract:** loaded a remote public dataset  
- **Transform:** aggregated data using pandas  
- **Load:** uploaded processed data into BigQuery  
- **Validate:** executed SQL query to verify stored data  

This notebook represents a clean, functional example of integrating Python with Google BigQuery for real ETL workflows.
